
# A/B Test Analysis – Two-Proportion Z-Test & Chi-Square Test

This notebook performs:
1. **Two-Proportion Z-Test** for conversion rate difference between `A (Control)` and `B (Treatment)`  
2. **Chi-Square Test of Independence** on the 2×2 contingency table  

We also include **confidence intervals**, **effect sizes** (absolute difference and relative risk), and concise **interpretation** at α = 0.05.

---


In [1]:

import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import chi2_contingency
from math import sqrt


## 1) Data Setup

In [2]:

# Given counts
sessions_A, conv_A = 4912, 127
sessions_B, conv_B = 4866, 78

# Build a small summary table
data_df = pd.DataFrame({
    "Variant": ["A (Control)", "B (Treatment)"],
    "Sessions": [sessions_A, sessions_B],
    "Conversions": [conv_A, conv_B],
    "Conversion Rate (%)": [100*conv_A/sessions_A, 100*conv_B/sessions_B]
})
data_df.round(4)


,Variant,Sessions,Conversions,Conversion Rate (%)
0,A (Control),4912,127,2.5855
1,B (Treatment),4866,78,1.6030


## 2) Two-Proportion Z-Test

In [3]:

# H0: p_A == p_B  vs  H1: p_A != p_B

count = np.array([conv_A, conv_B])
nobs  = np.array([sessions_A, sessions_B])
z_stat, pval_ztest = proportions_ztest(count, nobs, alternative="two-sided")

# 95% CI for difference (p_A - p_B) using Wald (unpooled) approximation
pA = conv_A / sessions_A
pB = conv_B / sessions_B
diff = pA - pB
z_975 = 1.959963984540054  # ~97.5th percentile for two-sided 95% CI
se_diff = sqrt(pA*(1-pA)/sessions_A + pB*(1-pB)/sessions_B)
ci_low = diff - z_975 * se_diff
ci_high = diff + z_975 * se_diff

ztest_df = pd.DataFrame({
    "Metric": ["Z statistic", "p-value (two-sided)", "p_A", "p_B", "Diff (p_A - p_B)", "95% CI Low", "95% CI High"],
    "Value": [z_stat, pval_ztest, pA, pB, diff, ci_low, ci_high]
})
ztest_df


,Metric,Value
0,Z statistic,3.390721
1,p-value (two-sided),0.000697
2,p_A,0.025855
3,p_B,0.016030
4,Diff (p_A - p_B),0.009825
5,95% CI Low,0.004155
6,95% CI High,0.015495


## 3) Chi-Square Test of Independence (2×2)

In [4]:

# Contingency table
contingency = np.array([
    [conv_A, sessions_A - conv_A],
    [conv_B, sessions_B - conv_B]
])

chi2_stat, pval_chi2, dof, expected = chi2_contingency(contingency, correction=False)

contingency_df = pd.DataFrame(contingency, 
                              index=["A (Control)", "B (Treatment)"], 
                              columns=["Conversion=Yes", "Conversion=No"])
expected_df = pd.DataFrame(expected, 
                           index=["A (Control)", "B (Treatment)"], 
                           columns=["Expected Yes", "Expected No"])

contingency_df, expected_df.round(2), pd.DataFrame({
    "Metric": ["Chi-square statistic", "Degrees of freedom", "p-value (two-sided)"],
    "Value": [chi2_stat, dof, pval_chi2]
})


(               Conversion=Yes  Conversion=No
 A (Control)               127           4785
 B (Treatment)              78           4788,
                Expected Yes  Expected No
 A (Control)          102.98      4809.02
 B (Treatment)        102.02      4763.98,
                  Metric      Value
 0  Chi-square statistic  11.496986
 1    Degrees of freedom   1.000000
 2   p-value (two-sided)   0.000697)

## 4) Practical Effect Size

In [5]:

# Absolute difference and Relative Risk (RR)
ARR = pA - pB
RR = pB / pA if pA > 0 else np.nan

# Standard error for log(RR) and 95% CI for RR
se_log_rr = sqrt((1/conv_B) - (1/sessions_B) + (1/conv_A) - (1/sessions_A))
log_rr = np.log(RR)
rr_ci_low = np.exp(log_rr - z_975 * se_log_rr)
rr_ci_high = np.exp(log_rr + z_975 * se_log_rr)

effects_df = pd.DataFrame({
    "Metric": ["Absolute difference (p_A - p_B)", "Relative Risk (B/A)", "RR 95% CI Low", "RR 95% CI High"],
    "Value": [ARR, RR, rr_ci_low, rr_ci_high]
})
effects_df


,Metric,Value
0,Absolute difference (p_A - p_B),0.009825
1,Relative Risk (B/A),0.619979
2,RR 95% CI Low,0.468968
3,RR 95% CI High,0.819617


## 5) Conclusions (α = 0.05)

In [7]:

alpha = 0.05
print(f"이표본 비율 z-검정 p-값 = {pval_ztest:.6f} -> "
      f"{'H0 기각 (비율이 다름)' if pval_ztest < alpha else 'H0 기각 실패'}")
print(f"카이제곱 검정 p-값 = {pval_chi2:.6f} -> "
      f"{'H0 기각 (연관성 있음)' if pval_chi2 < alpha else 'H0 기각 실패'}")

print()
print(f"추정된 전환율: p_A = {pA:.4%}, p_B = {pB:.4%}")
print(f"추정된 차이 (A - B): {diff:.4%}, 95% 신뢰구간 [{ci_low:.4%}, {ci_high:.4%}]")
print(f"상대위험도 (B/A): {RR:.3f}, 95% 신뢰구간 [{rr_ci_low:.3f}, {rr_ci_high:.3f}]")


이표본 비율 z-검정 p-값 = 0.000697 -> H0 기각 (비율이 다름)
카이제곱 검정 p-값 = 0.000697 -> H0 기각 (연관성 있음)

추정된 전환율: p_A = 2.5855%, p_B = 1.6030%
추정된 차이 (A - B): 0.9825%, 95% 신뢰구간 [0.4155%, 1.5495%]
상대위험도 (B/A): 0.620, 95% 신뢰구간 [0.469, 0.820]



---

### Notes
- The two-proportion z-test and chi-square test on a 2×2 table are asymptotically equivalent for large samples.
- We used **no Yates’ continuity correction** in `chi2_contingency(..., correction=False)` to align with the z-test.
- If counts are small, consider **Fisher’s exact test**.
